<img src="http://spark.apache.org/images/spark-logo-trademark.png" align="right">

PySpark and Data Movement Costs
=================

We've seen how Big Data collections like the PySpark RDD provide parallel and distributed versions of common operations.  These allow us to write distributed code similar to how we write sequential code.  However, while these operations may produce the same result, they also have different costs from what we might be used to.  Some operations that were previously fast may now be very slow.  Some operations that were slow may now be fast.

Fortunately there are often alternative algorithms to achieve the same results in faster time.  Understanding when to use these can greatly speed up our analyses.  In this notebook we look at two examples:

1.  Finding the largest elements of a collection of random numbers
2.  Performing a groupby-aggregate query on JSON records of GitHub data.

In each example we consider the performance of both a straightforward-and-slow approach, as well as introduce a less-straightforward but much faster approach.

*Note: there are expensive serialization costs moving from Python to JVM*

## Sorting and TopK with Random data

We create a large set of random numbers and store them as an RDD.  We find the largest numbers with two methods:

1.  Sort the RDD, then take the top five elements
2.  Call the `top` method

We find that calling the specialized `top` method is *much* faster than performing a full sort.  

*Note: had we used the spark dataframe API then Spark would have converted the first call into the second automatically.*

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand, randn
spark = SparkSession.builder.master('spark://schedulers:7077').getOrCreate()

In [ ]:
df = spark.range(0, 10000000, numPartitions=4)
df.rdd.getNumPartitions()

In [ ]:
# Create dataset 

random_df = df.select(rand(seed=10).alias("uniform"))
rdd = random_df.rdd.map(lambda x: x[0]).cache()
print(rdd.count())
print(rdd.take(5))

In [ ]:
%time rdd.sortBy(lambda t: t, ascending=False).take(5)

In [ ]:
%time rdd.top(5)

### ... with DataFrames

Spark dataframes are faster than Spark RDDs here for two reasons:

1.  It can do high-level query optimizations to turn `sort+take` into `top`.
2.  It can operate directly on efficient data structures rather than many small Python objects

In [ ]:
%time random_df.sort('uniform', ascending=False).take(5)

## Groupby-aggregate with Github JSON data

We learn the same lesson, that smarter algorithms can be much faster than the obvious approach, this time with real data.  

We read some JSON GitHub Data with Spark.  This includes every commit, comment, and pull request that occurred January 1st, 2015.

In [ ]:
df = spark.read.json("s3a://githubarchive-data/2015-01-01-*.json.gz")
df.take(2)

In [ ]:
# Load data into distributed memory
dfc = df.cache()
js = dfc.rdd
print(js.count(), js.take(1))

### Count the number of records, grouped by type

### ... with groupBy

In [ ]:
%%time
js.groupBy(lambda d: d['type']).map(lambda kv: (kv[0], len(kv[1]))).collect()

### ... with combineByKey

In [ ]:
%%time
def add(acc, x): return acc + 1
def global_add(x, y): return x + y

js.keyBy(lambda d: d['type']).combineByKey(lambda x: 1, add, global_add).collect()

<table>
    <tr>
      <td>
        <img src="https://databricks.gitbooks.io/databricks-spark-knowledge-base/content/images/group_by.png" width="400">
      </td>
      <td>
        <img src="https://databricks.gitbooks.io/databricks-spark-knowledge-base/content/images/reduce_by.png" width="400">
      </td>
    </tr>
</table>



[--Databricks](https://databricks.gitbooks.io/databricks-spark-knowledge-base/content/best_practices/prefer_reducebykey_over_groupbykey.html)

### ... with DataFrames

Again, Spark dataframes let us use straightforward syntax `groupby(...).count()` but rewrites our intent to the more efficient approach.

In [ ]:
%time dfc.groupBy(dfc['type']).count().collect()